In [1]:
import numpy as np
import tensorflow as tf
import keras

from keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score

import models_base, models_top, utils

2025-01-21 16:43:38.453691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 16:43:38.471877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737445418.484383  579032 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737445418.487627  579032 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 16:43:38.505672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets(
    "data/data_disruptive_feature_exclusion",
    val_split=0.2,
    )

# Extract features and labels from the dataset
x_train = np.concatenate([x for x, y in train_dataset], axis=0)
y_train = np.concatenate([y for x, y in train_dataset], axis=0)
x_val = np.concatenate([x for x, y in val_dataset], axis=0)
y_val = np.concatenate([y for x, y in val_dataset], axis=0)

Found 374 files belonging to 6 classes.
Using 300 files for training.
Using 74 files for validation.


In [10]:
# designing the model
def eV2L_midsize3(): # this can be adjusted to be hyper-tunable
    inputs = keras.Input(shape=(224, 224, 3))
    x = utils.image_preprocessing_v1()(inputs) # image preprocessing
    x = models_base.EV2L((224, 224, 3))(x, training=False) # pre-trained model
    x = models_top.midsize3()(x) # top model
    outputs = layers.Dense(6, activation='softmax', name="classifier")(x)

    model = keras.Model(inputs, outputs)
    
    model.compile(loss='categorical_crossentropy',
                   optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
                   metrics=['accuracy'])
    
    return model

eV2L_midsize3().summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ image_preprocessing_v1          │ (None, 224, 224, 3)    │             7 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-l (Functional)   │ (None, 7, 7, 1280)     │   117,746,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ basic_midsize3 (Functional)     │ (None, 256)            │     1,837,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,586,221 (456.19 MB)

 Trainable params: 1,839,366 (7.02 MB)

 Non-trainable params: 117,746,855 (449.17 MB)

In [11]:
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
fold_no = 1
acc_per_fold = []
loss_per_fold = []
f1_per_fold = []
precision_per_fold = []
recall_per_fold = []

for train_index, val_index in kf.split(x_train, y_train):
        print(f"Starting fold {fold_no}")

        x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        model = eV2L_midsize3()

        model.fit(x_train_fold, y_train_fold,
                batch_size=64,
                epochs=200,
                validation_data=(x_val_fold, y_val_fold),
                callbacks = [
                        utils.callback_earlystop(30),
                        utils.callback_savemodel(f"models/eV2L_midsize3_fold{fold_no}.keras")
        ])

        # Evaluate using numpy to calculate custom metrics
        y_pred = model.predict(x_val_fold)
        fold_f1 = utils.f1(y_val_fold, y_pred)
        fold_precision = utils.precision(y_val_fold, y_pred)
        fold_recall = utils.recall(y_val_fold, y_pred)
        scores = model.evaluate(x_val_fold, y_val_fold, verbose=0)

        print(f"Score for fold {fold_no:.2f}: Loss of {scores[0]:.2f}; Accuracy of {scores[1]*100:.2f}%; F1 of {fold_f1*100:.2f}%; Precision of {fold_precision*100:.2f}%; Recall of {fold_recall*100:.2f}%")

        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        f1_per_fold.append(fold_f1 * 100)
        precision_per_fold.append(fold_precision * 100)
        recall_per_fold.append(fold_recall * 100)
        fold_no += 1

Starting fold 1
Epoch 1/200


I0000 00:00:1737445692.525056  579149 cuda_dnn.cc:529] Loaded cuDNN version 90600


4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.1434 - loss: 1.8453 - val_accuracy: 0.1200 - val_loss: 1.8137
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 339ms/step - accuracy: 0.1855 - loss: 1.7873 - val_accuracy: 0.1200 - val_loss: 1.8282
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 951ms/step - accuracy: 0.2546 - loss: 1.7584 - val_accuracy: 0.2000 - val_loss: 1.7856
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 348ms/step - accuracy: 0.2355 - loss: 1.7369 - val_accuracy: 0.1200 - val_loss: 1.8172
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.2393 - loss: 1.7333 - val_accuracy: 0.2267 - val_loss: 1.7824
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 705ms/step - accuracy: 0.2286 - loss: 1.7424 - val_accuracy: 0.1867 - val_loss: 1.7841
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 367ms/step - accuracy: 0.2560 - loss: 1.7606 - val_accuracy: 0.1867 - val_loss: 1.7847
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.2062 - loss: 1.7303 - val_accuracy: 0.2000 - val_loss: 1.7950
E

In [13]:
print("Score per fold")

for i in range(0, len(acc_per_fold)):
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]:.2f} - Accuracy: {acc_per_fold[i]:.2f}% - F1: {f1_per_fold[i]:.2f}% - Precision: {precision_per_fold[i]:.2f}% - Recall: {recall_per_fold[i]:.2f}%')

print('\nAverage scores for all folds:')
print(f'> Accuracy:\t {np.mean(acc_per_fold):5.2f} (+- {np.std(acc_per_fold):.2f})')
print(f'> Loss:\t\t {np.mean(loss_per_fold):5.2f}')
print(f'> F1:\t\t {np.mean(f1_per_fold):5.2f} (+- {np.std(f1_per_fold):.2f})')
print(f'> Precision:\t {np.mean(precision_per_fold):5.2f} (+- {np.std(precision_per_fold):.2f})')
print(f'> Recall:\t {np.mean(recall_per_fold):5.2f} (+- {np.std(recall_per_fold):.2f})')

Score per fold
> Fold 1 - Loss: 1.85 - Accuracy: 16.00% - F1: 11.64% - Precision: 31.74% - Recall: 16.00%
> Fold 2 - Loss: 1.79 - Accuracy: 30.67% - F1: 24.55% - Precision: 22.38% - Recall: 30.67%
> Fold 3 - Loss: 1.87 - Accuracy: 16.00% - F1: 11.12% - Precision: 21.17% - Recall: 16.00%
> Fold 4 - Loss: 1.99 - Accuracy: 22.67% - F1: 16.73% - Precision: 27.48% - Recall: 22.67%

Average scores for all folds:
> Accuracy:	 21.33 (+- 6.04)
> Loss:		  1.87
> F1:		 16.01 (+- 5.40)
> Precision:	 25.69 (+- 4.22)
> Recall:	 21.33 (+- 6.04)


In [14]:
# training the model
eV2L_midsize3_history = eV2L_midsize3().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_midsize3.keras'),
        utils.callback_earlystop(30)
    ],
)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.1913 - loss: 1.8164

2025-01-21 16:59:33.852442: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 57142784 bytes after encountering the first element of size 57142784 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.1966 - loss: 1.8122 - val_accuracy: 0.2188 - val_loss: 1.7622
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 859ms/step - accuracy: 0.2994 - loss: 1.7152 - val_accuracy: 0.2188 - val_loss: 1.7499
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 926ms/step - accuracy: 0.3256 - loss: 1.6501 - val_accuracy: 0.3281 - val_loss: 1.6801
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 802ms/step - accuracy: 0.4122 - loss: 1.5922 - val_accuracy: 0.3438 - val_loss: 1.6484
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 755ms/step - accuracy: 0.4630 - loss: 1.5540 - val_accuracy: 0.3281 - val_loss: 1.6216
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 740ms/step - accuracy: 0.4915 - loss: 1.5179 - val_accuracy: 0.3750 - val_loss: 1.5644
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 768ms/step - accuracy: 0.5153 - loss: 1.4342 - val_accuracy: 0.4531 - val_loss: 1.4896
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 767ms/step - accuracy: 0.5108 - loss: 1.3962 - val_accuracy: 0.4688 - val_loss: 1.426